# Auxilary Output
You may want to add some auxiliary outputs in a NN architecture to ensure that the underlying part of the network learns something useful on its own, without relying on the rest of the network.(regularization technique)

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

X_train.shape[1:]

import tensorflow as tf
from tensorflow import keras

In [2]:
input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')
hidden1 = keras.layers.Dense(30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.Concatenate()([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)

In [3]:
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

In [4]:
model.compile(loss=['mse', 'mse'], loss_weights=[0.9, 0.1], optimizer="sgd")

In [5]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [6]:
history = model.fit(
    [X_train_A, X_train_B], [y_train, y_train], epochs=20,
    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid])
)

Epoch 1/20
363/363 [==============================] - 3s 4ms/step - loss: 0.9420 - main_output_loss: 0.7983 - aux_output_loss: 2.2359 - val_loss: 0.6771 - val_main_output_loss: 0.6221 - val_aux_output_loss: 1.1725
Epoch 2/20
363/363 [==============================] - 2s 4ms/step - loss: 0.5907 - main_output_loss: 0.5392 - aux_output_loss: 1.0542 - val_loss: 0.5877 - val_main_output_loss: 0.5428 - val_aux_output_loss: 0.9916
Epoch 3/20
363/363 [==============================] - 2s 6ms/step - loss: 1.2652 - main_output_loss: 1.2574 - aux_output_loss: 1.3355 - val_loss: 0.5625 - val_main_output_loss: 0.5243 - val_aux_output_loss: 0.9066
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5429 - main_output_loss: 0.5073 - aux_output_loss: 0.8625 - val_loss: 0.5056 - val_main_output_loss: 0.4725 - val_aux_output_loss: 0.8038
Epoch 5/20
363/363 [==============================] - 2s 6ms/step - loss: 0.5028 - main_output_loss: 0.4732 - aux_output_loss: 0.7690 - val_loss

In [7]:
total_loss, main_loss, _aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])
print(total_loss, main_loss, _aux_loss)

162/162 [==============================] - 1s 8ms/step - loss: 0.3786 - main_output_loss: 0.3617 - aux_output_loss: 0.5315
0.37864241003990173 0.36165574193000793 0.5315234065055847


In [9]:
y_pred_main, y_pred_max = model.predict([X_new_A, X_new_B])
print(y_pred_main, y_pred_max)

1/1 [==============================] - 1s 617ms/step
[[1.0947163]
 [1.8038183]
 [1.1701789]] [[1.0655814]
 [1.8611231]
 [1.0172672]]
